In [2]:
import os

print("当前 Input 目录下的所有文件：")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

当前 Input 目录下的所有文件：
/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv
/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg


In [3]:
import pandas as pd
import numpy as np

# --- 1. 定义 NSL-KDD 的列名 ---
# NSL-KDD 的 txt 文件没有表头，我们需要手动加上，否则第一行数据会变成列名
kdd_cols = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 
    'target', 'difficulty' # 最后一列通常是难度分数，训练时不需要，但读入时得占位
]

# --- 2. 加载数据 ---
print("正在加载 NSL-KDD...")
# 加载训练集
df_nsl_train = pd.read_csv('/kaggle/input/nslkdd/KDDTrain+.txt', names=kdd_cols)
# 加载测试集 (Day 5 验证基线时要用)
df_nsl_test = pd.read_csv('/kaggle/input/nslkdd/KDDTest+.txt', names=kdd_cols)

print(f"NSL-KDD Train: {df_nsl_train.shape}")
print(f"NSL-KDD Test:  {df_nsl_test.shape}")

print("\n正在加载 CIC-IDS-2017 (Cleaned)...")
# 读取你找到的 cleaned 版本
df_cic = pd.read_csv('/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv')

# 去除 CIC-IDS 列名的空格 (这是一个常见坑)
df_cic.columns = df_cic.columns.str.strip()

print(f"CIC-IDS Shape: {df_cic.shape}")

# --- 3. 标签统一与列处理 ---
print("\n正在统一标签列名...")

# 3.1 处理 NSL-KDD
# 删除 'difficulty' 列（这对分类任务没用）
if 'difficulty' in df_nsl_train.columns:
    df_nsl_train.drop('difficulty', axis=1, inplace=True)
if 'difficulty' in df_nsl_test.columns:
    df_nsl_test.drop('difficulty', axis=1, inplace=True)

# 确保标签叫 'target' (刚才 names 里已经指定了 target，这里确认一下)
print(f"NSL-KDD 标签列: {df_nsl_train.columns[-1]}")

# 3.2 处理 CIC-IDS
# CIC-IDS 清洗版的标签列通常叫 'Label'，我们把它改成 'target' 以便代码复用
if 'Label' in df_cic.columns:
    df_cic.rename(columns={'Label': 'target'}, inplace=True)

print(f"CIC-IDS 标签列: {df_cic.columns[-1]}")

# --- 4. CIC-IDS 最终清洗 (保险起见) ---
print("\n执行 CIC-IDS 脏数据检查...")

# 即使是 cleaned 版本，也可能在转换过程中产生 NaN
# 替换 Infinity
df_cic.replace([np.inf, -np.inf], np.nan, inplace=True)

# 检查空值数量
null_count = df_cic.isna().sum().sum()
if null_count > 0:
    print(f"警告：发现 {null_count} 个空值，正在删除...")
    df_cic.dropna(inplace=True)
else:
    print("完美！CIC-IDS 数据中没有 NaN 或 Infinity。")

print("-" * 30)
print("P1 任务完成。当前内存中包含：")
print("1. df_nsl_train (NSL-KDD 训练数据)")
print("2. df_nsl_test  (NSL-KDD 测试数据)")
print("3. df_cic       (CIC-IDS 全量数据)")

正在加载 NSL-KDD...
NSL-KDD Train: (125973, 43)
NSL-KDD Test:  (22544, 43)

正在加载 CIC-IDS-2017 (Cleaned)...
CIC-IDS Shape: (2520751, 53)

正在统一标签列名...
NSL-KDD 标签列: target
CIC-IDS 标签列: Attack Type

执行 CIC-IDS 脏数据检查...
完美！CIC-IDS 数据中没有 NaN 或 Infinity。
------------------------------
P1 任务完成。当前内存中包含：
1. df_nsl_train (NSL-KDD 训练数据)
2. df_nsl_test  (NSL-KDD 测试数据)
3. df_cic       (CIC-IDS 全量数据)


In [4]:
# 1. 查看 CIC-IDS 里的所有非数字列（确认除了标签外，还有没有其他列需要 One-Hot）
print("--- CIC-IDS 非数值列 ---")
print(df_cic.select_dtypes(include=['object']).columns.tolist())

# 2. 查看 CIC-IDS 标签列里到底有哪些值（确认正常流量是叫 'BENIGN' 还是 'Normal'）
# 注意：根据你刚才 P1 的反馈，列名可能还是 'Attack Type'，或者被改为 'target'
# 我们先试着找一下存在的那个
cic_label_col = 'target' if 'target' in df_cic.columns else 'Attack Type'
print(f"\n--- CIC-IDS 标签值分布 (前10个) ---")
print(df_cic[cic_label_col].value_counts().head(10))

# 3. 查看 NSL-KDD 标签列里到底有哪些值（确认正常流量是叫 'normal'）
print("\n--- NSL-KDD 标签值分布 (前10个) ---")
print(df_nsl_train['target'].value_counts().head(10))

--- CIC-IDS 非数值列 ---
['Attack Type']

--- CIC-IDS 标签值分布 (前10个) ---
Attack Type
Normal Traffic    2095057
DoS                193745
DDoS               128014
Port Scanning       90694
Brute Force          9150
Web Attacks          2143
Bots                 1948
Name: count, dtype: int64

--- NSL-KDD 标签值分布 (前10个) ---
target
normal         67343
neptune        41214
satan           3633
ipsweep         3599
portsweep       2931
smurf           2646
nmap            1493
back             956
teardrop         892
warezclient      890
Name: count, dtype: int64


In [5]:
import pandas as pd
import numpy as np
import gc # 内存回收工具

# --- 1. 处理 CIC-IDS (最简单，先搞定) ---
print("正在处理 CIC-IDS...")

# 1.1 强制重命名标签 (根据你的侦察结果，列名是 'Attack Type')
if 'Attack Type' in df_cic.columns:
    df_cic.rename(columns={'Attack Type': 'target'}, inplace=True)

# 1.2 标签二值化
# 规则：'Normal Traffic' -> 0 (正常), 其他所有 -> 1 (攻击)
df_cic['label_encoded'] = df_cic['target'].apply(lambda x: 0 if x == 'Normal Traffic' else 1)

print(f"CIC-IDS 处理完毕。正常样本数: {(df_cic['label_encoded']==0).sum()}")


# --- 2. 处理 NSL-KDD (需要 One-Hot) ---
print("\n正在处理 NSL-KDD...")

# 2.1 拼接 Train 和 Test
# 为了保证训练集和测试集的 One-Hot 列完全对齐（比如训练集有 http, 测试集也有 http）
df_nsl_train['source'] = 'train'
df_nsl_test['source'] = 'test'
df_combined = pd.concat([df_nsl_train, df_nsl_test], axis=0)

# 2.2 执行 One-Hot Encoding
# 针对三个已知的分类特征
cat_cols = ['protocol_type', 'service', 'flag']
print(f"正在对 {cat_cols} 进行 One-Hot 编码...")

df_combined_encoded = pd.get_dummies(df_combined, columns=cat_cols)

# 2.3 拆回 Train 和 Test
df_nsl_train_enc = df_combined_encoded[df_combined_encoded['source'] == 'train'].drop('source', axis=1)
df_nsl_test_enc = df_combined_encoded[df_combined_encoded['source'] == 'test'].drop('source', axis=1)

# 2.4 标签二值化
# 规则：'normal' -> 0 (正常), 其他所有 -> 1 (攻击)
df_nsl_train_enc['label_encoded'] = df_nsl_train_enc['target'].apply(lambda x: 0 if x == 'normal' else 1)
df_nsl_test_enc['label_encoded'] = df_nsl_test_enc['target'].apply(lambda x: 0 if x == 'normal' else 1)

print(f"NSL-KDD 处理完毕。")
print(f"Train 列数变化: {df_nsl_train.shape[1]} -> {df_nsl_train_enc.shape[1]}")


# --- 3. 清理内存 ---
# 删除不再需要的原始 DataFrame
del df_combined, df_combined_encoded
# 注意：暂时保留 df_nsl_train 原版，万一你想对比查看
gc.collect()

print("\n" + "="*30)
print("P2 任务执行成功！")
print(f"CIC-IDS 最终形状: {df_cic.shape} (含 label_encoded)")
print(f"NSL-KDD Train 最终形状: {df_nsl_train_enc.shape} (含 label_encoded)")
print("="*30)

正在处理 CIC-IDS...
CIC-IDS 处理完毕。正常样本数: 2095057

正在处理 NSL-KDD...
正在对 ['protocol_type', 'service', 'flag'] 进行 One-Hot 编码...
NSL-KDD 处理完毕。
Train 列数变化: 43 -> 124

P2 任务执行成功！
CIC-IDS 最终形状: (2520751, 54) (含 label_encoded)
NSL-KDD Train 最终形状: (125973, 124) (含 label_encoded)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import os

# ==========================================
# 0. 强力清洗函数 (新增)
# ==========================================
def clean_dataset_safely(df, dataset_name):
    print(f"[{dataset_name}] 执行深度清洗...")
    initial_shape = df.shape
    
    # 1. 强制转为数字 (处理潜在的 "Infinity" 字符串)
    # errors='coerce' 会把无法转数字的变成 NaN
    # 只对数值类型的列操作，避开 'target' 等字符串列
    num_cols = df.select_dtypes(include=[np.number]).columns
    # 如果有 float64 的列里混进了 object，这里强制转
    # 注意：为了效率，我们只对可能出问题的列操作，或者简单点全量检查
    # 这里直接处理所有非 object 列，确保万无一失
    for col in df.columns:
        if df[col].dtype != 'object':
            # 检查是否有无穷大
            if np.isinf(df[col]).any():
                print(f"  - 发现无穷大: 列 {col}")
                df[col] = df[col].replace([np.inf, -np.inf], np.nan)

    # 2. 再次删除 NaN
    df.dropna(inplace=True)
    
    # 3. 再次检查无穷大 (双重保险)
    # 仅检查数值列
    numeric_df = df.select_dtypes(include=[np.number])
    if np.isinf(numeric_df).values.sum() > 0:
        print("  - 警告：仍存在无穷大值！尝试强制替换...")
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)
        
    final_shape = df.shape
    print(f"[{dataset_name}] 清洗完成。删除行数: {initial_shape[0] - final_shape[0]}")
    return df

# ==========================================
# 1. 准备数据 & 应用 Log 变换
# ==========================================
print("正在准备数据...")

# --- 关键步骤：在切分前对 CIC-IDS 做最后一次核弹级清洗 ---
# 必须在分离 features 和 label 之前做，保证行对应关系
df_cic = clean_dataset_safely(df_cic, "CIC-IDS Full")


# --- 定义 Log 变换函数 ---
def apply_log_transform(df, dataset_name):
    cols_to_log = [c for c in df.columns if any(x in c.lower() for x in ['bytes', 'duration', 'count', 'packet'])]
    print(f"[{dataset_name}] 对 {len(cols_to_log)} 个特征应用 log1p...")
    df_log = df.copy()
    # 再次确保没有负数 (虽然很少见)
    # log1p 对负数会产生 NaN，这里做一个 clip 保护
    for col in cols_to_log:
        df_log[col] = df_log[col].clip(lower=0)
        
    df_log[cols_to_log] = df_log[cols_to_log].apply(np.log1p)
    return df_log

# --- 提取特征与标签 ---

# 1. NSL-KDD
nsl_drop = ['target', 'label_encoded']
df_nsl_features = df_nsl_train_enc.drop(nsl_drop, axis=1)
df_nsl_features = apply_log_transform(df_nsl_features, "NSL-KDD Train")
y_nsl_train_full = df_nsl_train_enc['label_encoded']

df_nsl_test_features = df_nsl_test_enc.drop(nsl_drop, axis=1)
df_nsl_test_features = apply_log_transform(df_nsl_test_features, "NSL-KDD Test")
y_nsl_test = df_nsl_test_enc['label_encoded']

# 2. CIC-IDS (已经清洗过)
cic_drop = ['target', 'label_encoded']
df_cic_features = df_cic.drop(cic_drop, axis=1)
df_cic_features = apply_log_transform(df_cic_features, "CIC-IDS")
y_cic_full = df_cic['label_encoded']


# ==========================================
# 2. P4: 数据集切分 (Split)
# ==========================================
print("\n正在执行数据集切分...")

# NSL-KDD
X_nsl_train, X_nsl_val, y_nsl_train, y_nsl_val = train_test_split(
    df_nsl_features, y_nsl_train_full, 
    test_size=0.1, random_state=42, stratify=y_nsl_train_full
)
X_nsl_test = df_nsl_test_features

# CIC-IDS
X_cic_temp, X_cic_test, y_cic_temp, y_cic_test = train_test_split(
    df_cic_features, y_cic_full, 
    test_size=0.2, random_state=42, stratify=y_cic_full
)
X_cic_train, X_cic_val, y_cic_train, y_cic_val = train_test_split(
    X_cic_temp, y_cic_temp, 
    test_size=0.125, random_state=42, stratify=y_cic_temp
)

print(f"切分完成。CIC-IDS Train: {X_cic_train.shape}")


# ==========================================
# 3. P3: 数值标准化 (MinMax Scaling)
# ==========================================
print("\n正在进行归一化 (MinMax Scaling)...")

scaler_nsl = MinMaxScaler()
X_nsl_train = scaler_nsl.fit_transform(X_nsl_train)
X_nsl_val   = scaler_nsl.transform(X_nsl_val)
X_nsl_test  = scaler_nsl.transform(X_nsl_test)

scaler_cic = MinMaxScaler()
# 这里如果再报错，那就是玄学了，因为我们在第一步已经清洗得非常彻底
X_cic_train = scaler_cic.fit_transform(X_cic_train)
X_cic_val   = scaler_cic.transform(X_cic_val)
X_cic_test  = scaler_cic.transform(X_cic_test)

print("归一化完成。")


# ==========================================
# 4. P5: 数据持久化 (Saving)
# ==========================================
print("\n正在保存最终数据...")
save_dir = '/kaggle/working/'

np.savez_compressed(
    os.path.join(save_dir, 'nsl_kdd_processed.npz'),
    X_train=X_nsl_train, y_train=y_nsl_train,
    X_val=X_nsl_val,     y_val=y_nsl_val,
    X_test=X_nsl_test,   y_test=y_nsl_test
)

np.savez_compressed(
    os.path.join(save_dir, 'cic_ids_processed.npz'),
    X_train=X_cic_train, y_train=y_cic_train,
    X_val=X_cic_val,     y_val=y_cic_val,
    X_test=X_cic_test,   y_test=y_cic_test
)

print("Day 4 任务（终极版）圆满结束！")

正在准备数据...
[CIC-IDS Full] 执行深度清洗...
[CIC-IDS Full] 清洗完成。删除行数: 0
[NSL-KDD Train] 对 7 个特征应用 log1p...
[NSL-KDD Test] 对 7 个特征应用 log1p...
[CIC-IDS] 对 27 个特征应用 log1p...

正在执行数据集切分...
切分完成。CIC-IDS Train: (1764525, 52)

正在进行归一化 (MinMax Scaling)...
归一化完成。

正在保存最终数据...
Day 4 任务（终极版）圆满结束！


In [7]:
import joblib
import os

save_dir = '/kaggle/working/'

print("正在补存 Scaler 对象...")

# 保存 NSL-KDD 的 scaler (包含 log 后的 min/max 信息)
joblib.dump(scaler_nsl, os.path.join(save_dir, 'scaler_nsl.pkl'))

# 保存 CIC-IDS 的 scaler
joblib.dump(scaler_cic, os.path.join(save_dir, 'scaler_cic.pkl'))

print(f"Scaler 对象已保存至: {save_dir}")
print("包含文件: scaler_nsl.pkl, scaler_cic.pkl")

正在补存 Scaler 对象...
Scaler 对象已保存至: /kaggle/working/
包含文件: scaler_nsl.pkl, scaler_cic.pkl


In [8]:
import pandas as pd

# 设置显示选项，确保所有列都能印出来，不被折叠
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

def audit_columns(df, name):
    print(f"\n======== {name} 数据列审计 ========")
    print(f"总行数: {df.shape[0]}, 总列数: {df.shape[1]}")
    print("-" * 80)
    print(f"{'列名':<30} | {'类型':<10} | {'唯一值数量':<10} | {'前5个样例值'}")
    print("-" * 80)
    
    for col in df.columns:
        # 获取该列的前5个非空值
        sample_vals = df[col].dropna().unique()[:5]
        # 获取类型
        dtype = str(df[col].dtype)
        # 获取唯一值数量
        n_unique = df[col].nunique()
        
        print(f"{col:<30} | {dtype:<10} | {n_unique:<10} | {sample_vals}")

# 检查内存中是否存在原始数据
# 如果你之前运行了 del df_cic, 这里可能需要你重新运行一下 P1 的加载代码
if 'df_cic' in locals():
    # 只需要看 CIC-IDS，因为 NSL-KDD 我们已经很清楚了 (protocol/service/flag)
    # 但为了保险，两个都看一眼
    audit_columns(df_cic, "CIC-IDS (Cleaned)")
else:
    print("错误：内存中找不到 df_cic。请重新运行 P1 (加载数据) 步骤后再运行此代码。")

if 'df_nsl_train' in locals():
    audit_columns(df_nsl_train, "NSL-KDD Train")


======== CIC-IDS (Cleaned) 数据列审计 ========
总行数: 2520751, 总列数: 54
--------------------------------------------------------------------------------
列名                             | 类型         | 唯一值数量      | 前5个样例值
--------------------------------------------------------------------------------
Destination Port               | int64      | 53791      | [   22 35396 60058 60060 35398]
Flow Duration                  | int64      | 1050854    | [1266342 1319353     160 1303488      77]
Total Fwd Packets              | int64      | 1413       | [41  1  2 15 19]
Total Length of Fwd Packets    | int64      | 17913      | [ 2664     0  2728  2071 18467]
Fwd Packet Length Max          | int64      | 5279       | [ 456    0 2065 2920   43]
Fwd Packet Length Min          | int64      | 384        | [ 0  6 48 45 34]
Fwd Packet Length Mean         | float64    | 99681      | [  64.97560976    0.           66.53658537 1035.5        1231.133333  ]
Fwd Packet Length Std          | float64    | 253866   

------------------------------------------------------------------------------------------
# 以下是重新进行的Day4的代码运行

In [9]:
import pandas as pd
import numpy as np
import os

# ==========================================
# 1. 定义清洗函数 (集成之前的修复经验)
# ==========================================
def clean_dataset_safely(df, dataset_name):
    print(f"[{dataset_name}] 正在执行深度清洗...")
    initial_shape = df.shape
    
    # 策略：
    # 1. 显式查找无穷大 (np.inf, -np.inf) 并替换为 NaN
    # 2. 针对 object 列之外的所有列，检查数值合法性
    
    # 仅选择数值类型的列进行检查
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    # 检查并替换 Inf
    if np.isinf(df[numeric_cols]).values.sum() > 0:
        print(f"  - 警告：发现数值列包含无穷大，正在替换...")
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        
    # 再次检查 NaN (包含原本的 NaN 和由 Inf 变来的 NaN)
    if df.isna().sum().sum() > 0:
        print(f"  - 发现空值 (NaN)，准备删除...")
        df.dropna(inplace=True)
        
    final_shape = df.shape
    print(f"[{dataset_name}] 清洗完成。删除行数: {initial_shape[0] - final_shape[0]}")
    return df

# ==========================================
# 2. 加载 NSL-KDD
# ==========================================
print("正在加载 NSL-KDD...")
kdd_cols = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 
    'target', 'difficulty'
]

# 路径根据你之前的运行记录
df_nsl_train = pd.read_csv('/kaggle/input/nslkdd/KDDTrain+.txt', names=kdd_cols)
df_nsl_test = pd.read_csv('/kaggle/input/nslkdd/KDDTest+.txt', names=kdd_cols)

# 删除 difficulty 列
if 'difficulty' in df_nsl_train.columns: df_nsl_train.drop('difficulty', axis=1, inplace=True)
if 'difficulty' in df_nsl_test.columns: df_nsl_test.drop('difficulty', axis=1, inplace=True)

print(f"NSL-KDD Train: {df_nsl_train.shape}")

# ==========================================
# 3. 加载 CIC-IDS & 深度清洗
# ==========================================
print("\n正在加载 CIC-IDS (Cleaned)...")
df_cic = pd.read_csv('/kaggle/input/cicids2017-cleaned-and-preprocessed/cicids2017_cleaned.csv')

# 去除列名空格
df_cic.columns = df_cic.columns.str.strip()

# 统一标签名：CIC-IDS 可能是 'Label' 或 'Attack Type'
if 'Label' in df_cic.columns:
    df_cic.rename(columns={'Label': 'target'}, inplace=True)
elif 'Attack Type' in df_cic.columns:
    df_cic.rename(columns={'Attack Type': 'target'}, inplace=True)

# 执行深度清洗
df_cic = clean_dataset_safely(df_cic, "CIC-IDS Full")

print("-" * 30)
print("P1 完成。所有数据集已加载并去除了脏数据。")

正在加载 NSL-KDD...
NSL-KDD Train: (125973, 42)

正在加载 CIC-IDS (Cleaned)...
[CIC-IDS Full] 正在执行深度清洗...
[CIC-IDS Full] 清洗完成。删除行数: 0
------------------------------
P1 完成。所有数据集已加载并去除了脏数据。


In [10]:
import pandas as pd
import numpy as np
import gc

# ==========================================
# 1. CIC-IDS 处理 (基于列审计结果)
# ==========================================
print("正在处理 CIC-IDS 特征...")

# [审计结论] 剔除 'Destination Port'，因为它有 53791 个唯一值，做数值处理无意义，做 One-Hot 会内存爆炸
if 'Destination Port' in df_cic.columns:
    print("  - 剔除 'Destination Port' 列 (避免维度爆炸)...")
    df_cic.drop('Destination Port', axis=1, inplace=True)

# 标签二值化：'Normal Traffic' -> 0, 其他 -> 1
# 注意：基于之前的 print 结果，正常标签是 'Normal Traffic'
df_cic['label_encoded'] = df_cic['target'].apply(lambda x: 0 if x == 'Normal Traffic' else 1)

print(f"CIC-IDS 处理完毕。特征数: {df_cic.shape[1]}")


# ==========================================
# 2. NSL-KDD 处理 (One-Hot)
# ==========================================
print("\n正在处理 NSL-KDD 特征...")

# 拼接 Train/Test 以保证 One-Hot 维度对齐
df_nsl_train['source'] = 'train'
df_nsl_test['source'] = 'test'
df_combined = pd.concat([df_nsl_train, df_nsl_test], axis=0)

# 指定需要编码的列 (NSL-KDD 的三个典型分类特征)
cat_cols = ['protocol_type', 'service', 'flag']
print(f"  - 执行 One-Hot 编码: {cat_cols}")
df_combined_encoded = pd.get_dummies(df_combined, columns=cat_cols)

# 拆分回 Train/Test
df_nsl_train_enc = df_combined_encoded[df_combined_encoded['source'] == 'train'].drop('source', axis=1)
df_nsl_test_enc = df_combined_encoded[df_combined_encoded['source'] == 'test'].drop('source', axis=1)

# 标签二值化：'normal' -> 0, 其他 -> 1
df_nsl_train_enc['label_encoded'] = df_nsl_train_enc['target'].apply(lambda x: 0 if x == 'normal' else 1)
df_nsl_test_enc['label_encoded'] = df_nsl_test_enc['target'].apply(lambda x: 0 if x == 'normal' else 1)

print(f"NSL-KDD 处理完毕。Train 维度: {df_nsl_train_enc.shape}")


# ==========================================
# 3. 内存清理
# ==========================================
del df_combined, df_combined_encoded
gc.collect()

print("\n" + "="*30)
print("P2 完成。分类特征已编码，标签已二值化。")
print(f"当前内存中的关键变量: df_cic, df_nsl_train_enc, df_nsl_test_enc")

正在处理 CIC-IDS 特征...
  - 剔除 'Destination Port' 列 (避免维度爆炸)...
CIC-IDS 处理完毕。特征数: 53

正在处理 NSL-KDD 特征...
  - 执行 One-Hot 编码: ['protocol_type', 'service', 'flag']
NSL-KDD 处理完毕。Train 维度: (125973, 124)

P2 完成。分类特征已编码，标签已二值化。
当前内存中的关键变量: df_cic, df_nsl_train_enc, df_nsl_test_enc


In [11]:
import numpy as np
import pandas as pd

# ==========================================
# 1. 定义 Log 变换函数
# ==========================================
def apply_log_transform(df, dataset_name):
    # 筛选包含 bytes, duration, count, packet 的列
    cols_to_log = [c for c in df.columns if any(x in c.lower() for x in ['bytes', 'duration', 'count', 'packet'])]
    print(f"[{dataset_name}] 正在对 {len(cols_to_log)} 个长尾特征应用 log1p 变换...")
    
    df_log = df.copy()
    # 保护性 Clip，防止极少数负数导致 NaN (虽然理论上不该有)
    for col in cols_to_log:
        df_log[col] = df_log[col].clip(lower=0)
    
    # log1p = log(x+1)
    df_log[cols_to_log] = df_log[cols_to_log].apply(np.log1p)
    return df_log

# ==========================================
# 2. 准备 NSL-KDD 数据
# ==========================================
print("正在提取 NSL-KDD 特征...")
nsl_drop = ['target', 'label_encoded']
# 如果 'source' 列还残留，也删掉
if 'source' in df_nsl_train_enc.columns: nsl_drop.append('source')

# 提取特征并应用 Log
X_nsl_train_raw = apply_log_transform(df_nsl_train_enc.drop(nsl_drop, axis=1), "NSL-KDD Train")
y_nsl_train_full = df_nsl_train_enc['label_encoded']

X_nsl_test_raw = apply_log_transform(df_nsl_test_enc.drop(nsl_drop, axis=1), "NSL-KDD Test")
y_nsl_test = df_nsl_test_enc['label_encoded']

# ==========================================
# 3. 准备 CIC-IDS 数据
# ==========================================
print("\n正在提取 CIC-IDS 特征...")
cic_drop = ['target', 'label_encoded']

X_cic_raw = apply_log_transform(df_cic.drop(cic_drop, axis=1), "CIC-IDS Full")
y_cic_full = df_cic['label_encoded']

print("-" * 30)
print("P3 (特征提取) 完成。")
print(f"NSL-KDD 特征维度: {X_nsl_train_raw.shape}")
print(f"CIC-IDS 特征维度: {X_cic_raw.shape}")

正在提取 NSL-KDD 特征...
[NSL-KDD Train] 正在对 7 个长尾特征应用 log1p 变换...
[NSL-KDD Test] 正在对 7 个长尾特征应用 log1p 变换...

正在提取 CIC-IDS 特征...
[CIC-IDS Full] 正在对 27 个长尾特征应用 log1p 变换...
------------------------------
P3 (特征提取) 完成。
NSL-KDD 特征维度: (125973, 122)
CIC-IDS 特征维度: (2520751, 51)


In [12]:
from sklearn.model_selection import train_test_split

print("正在执行数据集切分 (Stratified Split)...")

# ==========================================
# 1. NSL-KDD 切分
# ==========================================
# 原始 Train 拆分为: 真正的 Train (90%) + Validation (10%)
# 原始 Test 保持不变
X_nsl_train, X_nsl_val, y_nsl_train, y_nsl_val = train_test_split(
    X_nsl_train_raw, y_nsl_train_full, 
    test_size=0.1, random_state=42, stratify=y_nsl_train_full
)
# Test 集直接赋值
X_nsl_test = X_nsl_test_raw

# ==========================================
# 2. CIC-IDS 切分
# ==========================================
# 目标: Train(70%), Val(10%), Test(20%)

# 第一刀: 切出 20% 做最终测试集 (Test)
X_cic_temp, X_cic_test, y_cic_temp, y_cic_test = train_test_split(
    X_cic_raw, y_cic_full, 
    test_size=0.2, random_state=42, stratify=y_cic_full
)

# 第二刀: 从剩余 80% 中切出 12.5% (即总量的 10%) 做验证集 (Val)
X_cic_train, X_cic_val, y_cic_train, y_cic_val = train_test_split(
    X_cic_temp, y_cic_temp, 
    test_size=0.125, random_state=42, stratify=y_cic_temp
)

print("-" * 30)
print("P4 (数据切分) 完成。")
print(f"NSL-KDD: Train={X_nsl_train.shape}, Val={X_nsl_val.shape}, Test={X_nsl_test.shape}")
print(f"CIC-IDS: Train={X_cic_train.shape}, Val={X_cic_val.shape}, Test={X_cic_test.shape}")

正在执行数据集切分 (Stratified Split)...
------------------------------
P4 (数据切分) 完成。
NSL-KDD: Train=(113375, 122), Val=(12598, 122), Test=(22544, 122)
CIC-IDS: Train=(1764525, 51), Val=(252075, 51), Test=(504151, 51)


In [13]:
from sklearn.preprocessing import MinMaxScaler
import joblib
import os

print("正在进行归一化 (MinMax Scaling) 与 保存...")
save_dir = '/kaggle/working/'

# ==========================================
# 1. NSL-KDD 标准化 & 保存
# ==========================================
scaler_nsl = MinMaxScaler()
# Fit on Train
X_nsl_train = scaler_nsl.fit_transform(X_nsl_train)
# Transform others
X_nsl_val   = scaler_nsl.transform(X_nsl_val)
X_nsl_test  = scaler_nsl.transform(X_nsl_test)

# 保存 Scaler
joblib.dump(scaler_nsl, os.path.join(save_dir, 'scaler_nsl.pkl'))

# 保存数据
np.savez_compressed(
    os.path.join(save_dir, 'nsl_kdd_processed.npz'),
    X_train=X_nsl_train, y_train=y_nsl_train,
    X_val=X_nsl_val,     y_val=y_nsl_val,
    X_test=X_nsl_test,   y_test=y_nsl_test
)
print("  - NSL-KDD 处理完成并保存。")

# ==========================================
# 2. CIC-IDS 标准化 & 保存
# ==========================================
scaler_cic = MinMaxScaler()
# Fit on Train
X_cic_train = scaler_cic.fit_transform(X_cic_train)
# Transform others
X_cic_val   = scaler_cic.transform(X_cic_val)
X_cic_test  = scaler_cic.transform(X_cic_test)

# 保存 Scaler
joblib.dump(scaler_cic, os.path.join(save_dir, 'scaler_cic.pkl'))

# 保存数据
np.savez_compressed(
    os.path.join(save_dir, 'cic_ids_processed.npz'),
    X_train=X_cic_train, y_train=y_cic_train,
    X_val=X_cic_val,     y_val=y_cic_val,
    X_test=X_cic_test,   y_test=y_cic_test
)
print("  - CIC-IDS 处理完成并保存。")

print("=" * 30)
print("Day 4 任务圆满成功！")
print(f"文件输出目录: {save_dir}")
print("包含文件: nsl_kdd_processed.npz, cic_ids_processed.npz, scaler_nsl.pkl, scaler_cic.pkl")

正在进行归一化 (MinMax Scaling) 与 保存...
  - NSL-KDD 处理完成并保存。
  - CIC-IDS 处理完成并保存。
Day 4 任务圆满成功！
文件输出目录: /kaggle/working/
包含文件: nsl_kdd_processed.npz, cic_ids_processed.npz, scaler_nsl.pkl, scaler_cic.pkl
